In [83]:
import pandas as pd

def average_hamming(seqs):
    seqs = list(seqs)
    n = len(seqs)
    accumulative_mismatches = 0
    if n == 1:
        return 0
    for i in range(n-1):
        for j in range(i, n):
            accumulative_mismatches += mismatches(seqs[i],seqs[j])

    return accumulative_mismatches/n

def mismatches(s1,s2):
    n = 0
    for i in range(len(s1)):
        if s1[i] != s2[i]:
            n += 1

    return n

def phred_to_prob(phred):
    val = ord(phred) - 33
    return 10 ** -(val/10)

def overall_phred(qual_str):
    P = 1
    for q in qual_str:
        P *= 1 - phred_to_prob(q)

    return 1 - P

In [92]:
import math
def error_prob_to_phred33(p):
    """
    Convert an error probability to a Phred+33 ASCII character.

    Parameters:
        p (float): Error probability (between 0 and 1)

    Returns:
        str: Single-character Phred+33 quality symbol

    Raises:
        ValueError: If p is not between 0 and 1
    """
    if not (0 < p <= 1):
        raise ValueError("Error probability must be in (0, 1].")

    # Calculate Phred score
    phred = -10 * math.log10(p)
    phred = int(round(phred))

    # Clamp to typical Phred+33 range [0, 41]
    phred = max(0, min(phred, 41))
    return phred


def phred33_to_symbol(phred):
    # Convert to ASCII character
    return chr(phred + 33)

def error_prob_to_phred33_symbol(p):
    return phred33_to_symbol(error_prob_to_phred33(p))

In [94]:
print(phred_to_prob('A'))
print(error_prob_to_phred33(overall_phred('FFFFFFFFFF:')))

0.000630957344480193
23


In [91]:
(ord('8') - 33)/10


2.3

In [107]:
header = ['ID', 'cellbarcode', 'umi', 'viral', 'cellbarcode_quality', 'umi_quality', 'viral_quality', 'min_qual_cellbarcode', 'min_qual_umi', 'min_qual_virus']
wnv = pd.read_csv("wnv_mg4.tsv", delimiter="\t", names=header)

In [108]:
wnv

,ID,cellbarcode,umi,viral,cellbarcode_quality,umi_quality,viral_quality,min_qual_cellbarcode,min_qual_umi,min_qual_virus
0,A00405:682:HV27LDSX5:1:1101:15700:1282,TTCGAAGTCATGCAA,CAGGGGTGTGC,CTGACAGTTACCGTTACAGCAGCCACGCTTCTG,FFFFFFFFFFFFFFF,FFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,37.0,37.0,37.0
1,A00405:682:HV27LDSX5:1:1101:9182:4679,GGTGCGCTCTAATTC,TGGGACGTCCG,CTAACCGTCACAGTAACCGCAGCTACCCTGCTG,F:FFF:FFFFFFFFF,FFFFFF:FFFF,"FFFFFFFFFFF:FF,FFFFFFFFF:FFFFFFF:",25.0,25.0,11.0
2,A00405:682:HV27LDSX5:1:1101:24080:18082,TACGGGCTTCTCGTA,CTGTCTTCACA,CTCACTGTTACCGTCACTGCAGCTACCCTGCTC,:FFFFFFFFFFFFFF,FFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFF:FFFFFFFF:FFF,25.0,37.0,25.0
3,A00405:682:HV27LDSX5:1:1101:3739:21183,CGTTGGGTGTGGTGC,GCTCTAATTCT,CTGACAGTAACCGTTACAGCCGCTACTCTGCTT,"F,FFF:FFFF,FF,F",F:FF:FFFFFF,"F:,FFF,F,FF,F,FFF,:FFFFFFF::F,F:F",11.0,25.0,11.0
4,A00405:682:HV27LDSX5:1:1101:13883:26819,GGTGCGTTCATCACC,CTCTCTCTAGC,CTTACAGTTACCGTTACAGCAGCCACACTCCTG,FFFFFFFFFFFF:FF,FFFFFFFFFF:,"FFF,FFFFFFFFFFF,FFFFFFFFFFFFFFFFF",25.0,25.0,11.0
...,...,...,...,...,...,...,...,...,...,...
697,A00405:682:HV27LDSX5:1:1172:20844:2080,GACTACACTGACTAC,ATGATGGGTGT,CTCACCGTTACTGTCACTGCAGCAACCCTACTT,FFFFFFFFFFFFFFF,FFFFFFFFFFF,FFFFFFFFFFFFFFF:FFFF:FFF:FFFF:F:F,37.0,37.0,25.0
698,A00405:682:HV27LDSX5:1:1172:10484:3959,TGACAACTCATTTGG,GTTGTGGTGCG,CTCACAGTGACTGTAACAGCAGCTACGCTGCTA,FFFFFFF:FFFFFFF,FFFFF:FFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,25.0,25.0,37.0
699,A00405:682:HV27LDSX5:1:1172:25807:11710,TACTTACCATTCACT,TTTAACTATTC,CTTACCGTTACTGTGACTGCGGCGACACTCCTT,FFFFFFFF:::FFFF,"FFFFFFF,FFF","FFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFF",25.0,11.0,11.0
700,A00405:682:HV27LDSX5:1:1172:28800:22999,GAACCTATCTTGCCG,TTCTTTTGTTG,CTCACCGTTACTGTCACTGCAGCAACCCTACTT,FFFFFFFFFFFFFFF,FFFF:FFFFF:,"FFFFFFFFFFF:FFF,FFFFFFFFFFFFFFF,F",37.0,25.0,11.0


In [109]:
wnv['overall_quality'] = wnv[['min_qual_cellbarcode','min_qual_umi','min_qual_virus']].min(axis=1)

In [110]:
wnv

,ID,cellbarcode,umi,viral,cellbarcode_quality,umi_quality,viral_quality,min_qual_cellbarcode,min_qual_umi,min_qual_virus,overall_quality
0,A00405:682:HV27LDSX5:1:1101:15700:1282,TTCGAAGTCATGCAA,CAGGGGTGTGC,CTGACAGTTACCGTTACAGCAGCCACGCTTCTG,FFFFFFFFFFFFFFF,FFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,37.0,37.0,37.0,37.0
1,A00405:682:HV27LDSX5:1:1101:9182:4679,GGTGCGCTCTAATTC,TGGGACGTCCG,CTAACCGTCACAGTAACCGCAGCTACCCTGCTG,F:FFF:FFFFFFFFF,FFFFFF:FFFF,"FFFFFFFFFFF:FF,FFFFFFFFF:FFFFFFF:",25.0,25.0,11.0,11.0
2,A00405:682:HV27LDSX5:1:1101:24080:18082,TACGGGCTTCTCGTA,CTGTCTTCACA,CTCACTGTTACCGTCACTGCAGCTACCCTGCTC,:FFFFFFFFFFFFFF,FFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFF:FFFFFFFF:FFF,25.0,37.0,25.0,25.0
3,A00405:682:HV27LDSX5:1:1101:3739:21183,CGTTGGGTGTGGTGC,GCTCTAATTCT,CTGACAGTAACCGTTACAGCCGCTACTCTGCTT,"F,FFF:FFFF,FF,F",F:FF:FFFFFF,"F:,FFF,F,FF,F,FFF,:FFFFFFF::F,F:F",11.0,25.0,11.0,11.0
4,A00405:682:HV27LDSX5:1:1101:13883:26819,GGTGCGTTCATCACC,CTCTCTCTAGC,CTTACAGTTACCGTTACAGCAGCCACACTCCTG,FFFFFFFFFFFF:FF,FFFFFFFFFF:,"FFF,FFFFFFFFFFF,FFFFFFFFFFFFFFFFF",25.0,25.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
697,A00405:682:HV27LDSX5:1:1172:20844:2080,GACTACACTGACTAC,ATGATGGGTGT,CTCACCGTTACTGTCACTGCAGCAACCCTACTT,FFFFFFFFFFFFFFF,FFFFFFFFFFF,FFFFFFFFFFFFFFF:FFFF:FFF:FFFF:F:F,37.0,37.0,25.0,25.0
698,A00405:682:HV27LDSX5:1:1172:10484:3959,TGACAACTCATTTGG,GTTGTGGTGCG,CTCACAGTGACTGTAACAGCAGCTACGCTGCTA,FFFFFFF:FFFFFFF,FFFFF:FFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,25.0,25.0,37.0,25.0
699,A00405:682:HV27LDSX5:1:1172:25807:11710,TACTTACCATTCACT,TTTAACTATTC,CTTACCGTTACTGTGACTGCGGCGACACTCCTT,FFFFFFFF:::FFFF,"FFFFFFF,FFF","FFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFF",25.0,11.0,11.0,11.0
700,A00405:682:HV27LDSX5:1:1172:28800:22999,GAACCTATCTTGCCG,TTCTTTTGTTG,CTCACCGTTACTGTCACTGCAGCAACCCTACTT,FFFFFFFFFFFFFFF,FFFF:FFFFF:,"FFFFFFFFFFF:FFF,FFFFFFFFFFFFFFF,F",37.0,25.0,11.0,11.0


In [111]:
result = wnv.groupby("cellbarcode").agg(
        ave_mismatch_wnv = ("viral", average_hamming),
        n_seqs = ("viral", "count"),
        min_qual = ("overall_quality", "min")
    ).reset_index()
result

,cellbarcode,ave_mismatch_wnv,n_seqs,min_qual
0,AAACCTGCAGAGCCA,0.0,1,25.0
1,AAACCTGTCCAACTT,0.0,1,11.0
2,AAACGGGTTCACTAC,0.0,1,11.0
3,AAAGATGGTTTACTC,0.0,1,37.0
4,AAAGCAATCATGCAT,0.0,1,11.0
...,...,...,...,...
491,TTGTAGGTCCAAATG,0.0,1,25.0
492,TTTACTGCATGATGG,3.0,2,37.0
493,TTTCCTCCCAAAGCG,0.0,1,37.0
494,TTTGCGCTCTAATTC,4.5,2,11.0


In [112]:
top_candidates = result[(result["ave_mismatch_wnv"] > 0) & (result["min_qual"] >= 25)].sort_values(by="min_qual", ascending=False)
top_candidates

,cellbarcode,ave_mismatch_wnv,n_seqs,min_qual
492,TTTACTGCATGATGG,3.000000,2,37.0
28,ACATGGGTGTGGTGC,5.000000,2,25.0
101,ATCCAACTTTCTTCT,5.000000,2,25.0
83,AGCTCTAATTCTGGG,3.500000,2,25.0
310,GCTGGGTGTGGTGCG,13.000000,4,25.0
334,GGGTGTGGTGCGCTC,5.333333,3,25.0
367,GTCTAACTTTCTTCT,7.333333,3,25.0
354,GTCACAACTTTCTTC,6.666667,3,25.0
379,GTGTGGTGCGCTCTA,3.500000,2,25.0
419,TCACAGACGGGTTCA,4.000000,2,25.0


In [113]:
top_qual_barcodes = list(top_candidates['cellbarcode'])

In [114]:
top_qual_barcodes

['TTTACTGCATGATGG',
 'ACATGGGTGTGGTGC',
 'ATCCAACTTTCTTCT',
 'AGCTCTAATTCTGGG',
 'GCTGGGTGTGGTGCG',
 'GGGTGTGGTGCGCTC',
 'GTCTAACTTTCTTCT',
 'GTCACAACTTTCTTC',
 'GTGTGGTGCGCTCTA',
 'TCACAGACGGGTTCA',
 'TGCGTGGTGCGCTCT',
 'TGTCCAACTTTCTTC']

In [115]:
all_top_qual = wnv[wnv["cellbarcode"].isin(top_qual_barcodes)].copy()

In [116]:
all_top_qual["cellbarcode"] = pd.Categorical(
    all_top_qual["cellbarcode"],
    categories=top_qual_barcodes,
    ordered=True
)

## Order by quality score and evaluate viral divergence

In [117]:
all_top_qual[["cellbarcode","umi","viral","overall_quality"]].sort_values("cellbarcode").reset_index().head(20)

,index,cellbarcode,umi,viral,overall_quality
0,634,TTTACTGCATGATGG,GTGTGGTGCGC,CTCACAGTCACAGTAACTGCTGCTACGCTCCTG,37.0
1,609,TTTACTGCATGATGG,GTGTGGTGCGC,CTGACCGTCACTGTAACTGCCGCTACTCTCCTT,37.0
2,657,ACATGGGTGTGGTGC,GCTCTAATTCT,CTTACCGTCACCGTAACCGCAGCCACTCTACTC,37.0
3,484,ACATGGGTGTGGTGC,GCTCTAATTCT,CTAACTGTTACGGTCACTGCAGCGACACTCCTA,25.0
4,618,ATCCAACTTTCTTCT,GCATGATGGGT,CTTACCGTTACTGTGACTGCGGCGACACTCCTT,25.0
5,328,ATCCAACTTTCTTCT,GCATGATGGGT,CTCACAGTGACCGTCACAGCCGCCACTCTCCTA,25.0
6,320,AGCTCTAATTCTGGG,ACGTCCGTGGC,CTCACCGTTACTGTCACTGCAGCAACCCTACTT,25.0
7,569,AGCTCTAATTCTGGG,ACGTCCGTGGC,CTCACAGTCACAGTAACCGCTGCAACGCTACTT,25.0
8,133,GCTGGGTGTGGTGCG,CTCTAATTCTG,CTCACAGTGACTGTAACAGCAGCTACGCTGCTA,25.0
9,492,GCTGGGTGTGGTGCG,CTCTAATTCTG,CTTACCGTTACTGTGACTGCGGCGACACTCCTT,37.0


## Verify the divergence of the viral sequences sharing the same cell bar code and UMI

In [118]:
x = all_top_qual[["cellbarcode","umi","viral","overall_quality"]].sort_values("cellbarcode").reset_index()
x.groupby(["cellbarcode", "umi"], observed=True).agg(
    ave_mismatch_wnv=("viral", average_hamming),
    n_seqs=("viral", "count")
).sort_values(by="cellbarcode", ascending=True)
   

,,ave_mismatch_wnv,n_seqs
cellbarcode,umi,,
TTTACTGCATGATGG,GTGTGGTGCGC,3.000000,2
ACATGGGTGTGGTGC,GCTCTAATTCT,5.000000,2
ATCCAACTTTCTTCT,GCATGATGGGT,5.000000,2
AGCTCTAATTCTGGG,ACGTCCGTGGC,3.500000,2
GCTGGGTGTGGTGCG,CTCTAATTCTG,13.000000,4
GGGTGTGGTGCGCTC,TAATTCTGGGA,5.333333,3
GTCTAACTTTCTTCT,GCATGATGGGT,7.333333,3
GTCACAACTTTCTTC,TGCATGATGGG,6.666667,3
GTGTGGTGCGCTCTA,ATTCTGGGACG,3.500000,2
